# Generowanie i agregacja danych z wykorzystaniem biblioteki Faker w Pythonie
<br>


Poznaj możliwości biblioteki Faker w Pythonie (https://faker.readthedocs.io/) do dynamicznego generowania danych!

Niezależnie od tego, czy jesteś analitykiem danych, inżynierem czy specjalistą ds. danych, ten samouczek przeprowadzi Cię przez proces tworzenia realistycznych i zróżnicowanych zbiorów danych przy użyciu Faker.

# Instalacja Faker

Moduł Pythona `faker` wymaga instalacji. 

In [ ]:
!pip install faker

# Generowanie ramki danych Pandas z fałszywymi danymi

Zaimportuj `Faker` i ustaw ziarno losowości ($42$).

In [ ]:
from faker import Faker
# Set the seed value of the shared `random.Random` object
# across all internal generators that will ever be created
Faker.seed(42)

`fake` jest generatorem fałszywych danych z lokalizacją `pl_PL` (polska).

In [ ]:
fake = Faker('pl_PL')
fake.seed_locale('pl_PL', 42)
# Creates and seeds a unique `random.Random` object for
# each internal generator of this `Faker` instance
fake.seed_instance(42)

Za pomocą `fake` możesz generować fałszywe dane, takie jak imię i nazwisko, e-mail itp.

In [ ]:
print(f"A fake name: {fake.name()}")
print(f"A fake email: {fake.email()}")

Zaimportuj Pandas, aby zapisać dane do ramki danych

In [ ]:
import pandas as pd

Funkcja `create_row_faker` tworzy jeden wiersz fałszywych danych. W tym przypadku generujemy wiersz zawierający następujące pola:
 - `fake.name()` - imię i nazwisko
 - `fake.postcode()` - kod pocztowy
 - `fake.email()` - adres e-mail
 - `fake.country()` - kraj

In [ ]:
def create_row_faker(num=1):
    fake = Faker('pl_PL')
    fake.seed_locale('pl_PL', 42)
    fake.seed_instance(42)
    output = [{"name": fake.name(),
               "age": fake.random_int(0, 100),
               "postcode": fake.postcode(),
               "email": fake.email(),
               "nationality": fake.country(),
              } for x in range(num)]
    return output

Wygeneruj pojedynczy wiersz

In [ ]:
create_row_faker()

Wygeneruj `n=3` wiersze

In [ ]:
create_row_faker(3)

Wygeneruj ramkę danych `df_fake` z 5000 wierszy przy użyciu `create_row_faker`.

Używamy _magii komórki_ `%%time` do zmierzenia czasu operacji.

In [ ]:
%%time
df_fake = pd.DataFrame(create_row_faker(5000))

Wyświetl ramkę danych

In [ ]:
df_fake

Więcej generatorów fałszywych danych znajdziesz w [standardowych providerach](https://faker.readthedocs.io/en/master/providers.html#standard-providers) Faker oraz w [providerach społecznościowych](https://faker.readthedocs.io/en/master/communityproviders.html#community-providers).

## Kontrolowane anomalie (do testów walidacji)

Dodajemy kontrolowane anomalie do ramki danych `df_fake` w celu przetestowania walidacji danych. W tym przypadku wprowadzamy następujące anomalie:
- W 1% wierszy ustawiamy `email` na `None` 
- W 1% wierszy ustawiamy `postcode` na pusty ciąg znaków `""`



In [ ]:
import numpy as np

num_rows = df_fake.shape[0]
num_anomalies = int(0.01 * num_rows)
anomaly_indices_email = np.random.choice(num_rows, num_anomalies, replace=False)
anomaly_indices_postcode = np.random.choice(num_rows, num_anomalies, replace=False)
df_fake.loc[anomaly_indices_email, 'email'] = None
df_fake.loc[anomaly_indices_postcode, 'postcode'] = ""
df_fake

Możemy teraz sprawdzić, ile mamy brakujących wartości w kolumnie `email` oraz ile pustych wartości w kolumnie `postcode`.

In [ ]:
#Możemy teraz sprawdzić, ile mamy brakujących wartości w kolumnie `email` oraz ile pustych wartości w kolumnie `postcode`.
missing_emails = df_fake['email'].isnull().sum()
empty_postcodes = (df_fake['postcode'] == "").sum()
print(f"Number of missing emails: {missing_emails}")
print(f"Number of empty postcodes: {empty_postcodes}")

Możemy też wprowadzić anomalie podczas generowania danych w funkcji `create_row_faker`.

In [ ]:

def create_row_faker_with_anomalies(num=1, anomaly_rate=0.01):
    fake = Faker('pl_PL')
    fake.seed_locale('pl_PL', 42)
    fake.seed_instance(42)
    output = []
    for x in range(num):
        # Decide anomalies first so we avoid calling Faker for anomalous fields
        is_email_anomaly = np.random.rand() < anomaly_rate
        is_postcode_anomaly = np.random.rand() < anomaly_rate

        row = {
            "name": fake.name(),
            "age": fake.random_int(0, 100),
            "postcode": "" if is_postcode_anomaly else fake.postcode(),
            "email": None if is_email_anomaly else fake.email(),
            "nationality": fake.country(),
        }
        output.append(row)
    return output
df_fake_with_anomalies = pd.DataFrame(create_row_faker_with_anomalies(5000, anomaly_rate=0.01))

print(f"Number of data rows: {len(df_fake_with_anomalies)}")

missing_emails = df_fake_with_anomalies['email'].isnull().sum()
empty_postcodes = (df_fake_with_anomalies['postcode'] == "").sum()
print(f"Number of missing emails: {missing_emails}")
print(f"Number of empty postcodes: {empty_postcodes}")